In [35]:
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data.dataset import Dataset 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms,models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
import shutil
from sklearn.metrics import f1_score   

In [36]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [37]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])

In [38]:
transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [39]:
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [40]:
train_dir = "Data/train/"
test_dir = "Data/test/"
val_dir= "Data/val/"
train_flooded_dir = "Data/train/flooded/"
train_non_flooded_dir = "Data/train/non-flooded/"
val_flooded_dir = "Data/val/flooded/"
val_non_flooded_dir = "Data/val/non-flooded/"
test_flooded_dir = "Data/test/flooded/"
test_non_flooded_dir = "Data/test/non-flooded/"

In [41]:
train_dataset = ImageFolder(train_dir, transforms_train)
test_dataset = ImageFolder(test_dir, transforms_test)
val_dataset = ImageFolder(val_dir, transforms_val)
train_dataloader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_dataset, batch_size=12, shuffle=True, num_workers=8)
test_dataloader = DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=8)

In [42]:
print('Train dataset size:', len(train_dataset))
print('Test dataset size:', len(test_dataset))
print('Val dataset size:', len(val_dataset))
class_names = train_dataset.classes
print('Class names:', class_names)

Train dataset size: 644
Test dataset size: 140
Val dataset size: 138
Class names: ['flooded', 'non-flooded']


In [43]:
model = models.resnet18(pretrained=True)   #load resnet18 model
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Linear(num_features, 2) #(num_of_class == 2)
model = model.to(device) 
criterion = nn.CrossEntropyLoss()  #(set loss function)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

c:\Users\rorod\miniforge3\envs\new_python_nlp_2\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\rorod\miniforge3\envs\new_python_nlp_2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
num_epochs = 10   #(set no of epochs)
start_time = time.time() #(for showing time)

for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    targets = []
    results = []
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        results.append(preds)
        targets.append(labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    results =torch.cat(results, dim=0)
    targets = torch.cat(targets, dim=0)

    results = results.to('cpu').numpy().flatten()
    targets = targets.to('cpu').numpy().flatten()

    f1_value =  f1_score(results, targets)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))
    print('F1-Score', f1_value)
    """ Val Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        targets = []
        results = []
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            results.append(preds)
            targets.append(labels)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects / len(val_dataset) * 100.
        results = torch.cat(results, dim=0)
        targets =torch.cat(targets, dim=0)
        results = results.to('cpu').numpy().flatten()
        targets = targets.to('cpu').numpy().flatten()
        f1_value =  f1_score(results, targets)
        print('[Val #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))
        print('F1-Score', f1_value)

Epoch 0 running
[Train #0] Loss: 0.4320 Acc: 78.1056% Time: 70.0046s
F1-Score 0.7765451664025357
[Val #0] Loss: 0.1468 Acc: 94.9275% Time: 80.0055s
F1-Score 0.951048951048951
Epoch 1 running
[Train #1] Loss: 0.2395 Acc: 90.3727% Time: 148.0924s
F1-Score 0.9040247678018576
[Val #1] Loss: 0.1133 Acc: 97.1014% Time: 158.4764s
F1-Score 0.9714285714285714
Epoch 2 running
[Train #2] Loss: 0.2164 Acc: 90.8385% Time: 223.6860s
F1-Score 0.9079563182527302
[Val #2] Loss: 0.1418 Acc: 94.9275% Time: 231.7967s
F1-Score 0.951048951048951
Epoch 3 running
[Train #3] Loss: 0.1638 Acc: 94.5652% Time: 294.0051s
F1-Score 0.9455676516329704
[Val #3] Loss: 0.0790 Acc: 97.8261% Time: 301.1646s
F1-Score 0.9784172661870504
Epoch 4 running
[Train #4] Loss: 0.1939 Acc: 92.0807% Time: 363.9090s
F1-Score 0.9216589861751152
[Val #4] Loss: 0.0660 Acc: 97.1014% Time: 371.2928s
F1-Score 0.9710144927536232
Epoch 5 running
[Train #5] Loss: 0.0983 Acc: 96.1180% Time: 435.7382s
F1-Score 0.9614791987673343
[Val #5] Loss: 0

In [46]:
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        targets = []
        results = []
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            results.append(preds)
            targets.append(labels)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(test_dataset)
        epoch_acc = running_corrects / len(test_dataset) * 100.
        results = torch.cat(results, dim=0)
        targets =torch.cat(targets, dim=0)
        results = results.to('cpu').numpy().flatten()
        targets = targets.to('cpu').numpy().flatten()
        f1_value =  f1_score(results, targets)
        print('F1-Score', f1_value)
        print('[Val #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))

F1-Score 0.9655172413793104
[Val #9] Loss: 0.0841 Acc: 96.4286% Time: 777.8949s
